In [ ]:
# Experiment 7: Using pre-trained VGG16 for image classification

import os
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

print("TensorFlow version:", tf.__version__)

# -----------------------------
# 1. Basic configuration
# -----------------------------
img_height = 150
img_width = 150
batch_size = 32

train_dir = "data/train"         # change this
val_dir   = "data/validation"    # change this

# Automatically count number of classes from folders
num_classes = len(os.listdir(train_dir))
print("Number of classes:", num_classes)

# -----------------------------
# 2. Data generators
# -----------------------------
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

print("Class indices:", train_generator.class_indices)

# -----------------------------
# 3. Instantiate VGG16 base
# -----------------------------
conv_base = VGG16(
    weights='imagenet',
    include_top=False,                 # remove dense classifier of VGG16
    input_shape=(img_height, img_width, 3)
)

print(conv_base.summary())

# Freeze convolutional base for feature extraction
conv_base.trainable = False

# -----------------------------
# 4. Build the full model
# -----------------------------
model = models.Sequential([
    conv_base,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    metrics=['accuracy']
)

print(model.summary())

# -----------------------------
# 5. Train (Feature Extraction)
# -----------------------------
epochs = 10   # change as needed
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator
)

# -----------------------------
# 6. Plot accuracy and loss
# -----------------------------
acc      = history.history['accuracy']
val_acc  = history.history['val_accuracy']
loss     = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(1, len(acc) + 1)

plt.figure()
plt.plot(epochs_range, acc, label='Train Accuracy')
plt.plot(epochs_range, val_acc, label='Val Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')
plt.show()

plt.figure()
plt.plot(epochs_range, loss, label='Train Loss')
plt.plot(epochs_range, val_loss, label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')
plt.show()

# -----------------------------
# 7. OPTIONAL: Fine-tuning
# -----------------------------
# Unfreeze some top layers of VGG16
conv_base.trainable = True

# Let's say we fine-tune from block5_conv1 onward
fine_tune_from = None
for i, layer in enumerate(conv_base.layers):
    if layer.name == 'block5_conv1':
        fine_tune_from = i
        break

for i, layer in enumerate(conv_base.layers):
    if i < fine_tune_from:
        layer.trainable = False
    else:
        layer.trainable = True

model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),  # smaller LR
    metrics=['accuracy']
)

fine_tune_epochs = 5  # extra epochs for fine-tuning

history_fine = model.fit(
    train_generator,
    epochs=fine_tune_epochs,
    validation_data=validation_generator
)

# You can again plot accuracy & loss for fine-tuning if needed.


TensorFlow version: 2.19.0


FileNotFoundError: [Errno 2] No such file or directory: 'data/train'